<a href="https://colab.research.google.com/github/Rohan020400/Portfolio/blob/main/CameraToAudioFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 10 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,024 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [3]:
!pip install pytesseract

In [4]:
from PIL import Image
import pytesseract

In [5]:
!pip install -U Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.1.0 which is incompatible.


In [6]:
!pip install gTTS

In [7]:
!sudo apt install tesseract-ocr
!pip install opencv-python pytesseract Pillow gTTS

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


In [8]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
import pytesseract
from gtts import gTTS
import IPython.display as ipd
import io

# Install necessary packages
#!sudo apt install tesseract-ocr
#!pip install pytesseract Pillow gtts

# Configure Tesseract OCR path
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

def take_photo(quality=0.8 ):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({ 'video': true });

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')

    display(js)

    # Use the function to capture a photo.
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])

    # Create a NumPy array from the binary data
    image_array = np.frombuffer(binary, dtype=np.uint8)

    # Decode the NumPy array to an OpenCV image
    cv_image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    return cv_image

# Initialize the video capture (for real-time text detection)
cap = cv2.VideoCapture(0)
# Set camera resolution (adjust values as needed)
cap.set(3, 960)  # Width
cap.set(4, 600)   # Height
cap.set(cv2.CAP_PROP_FPS, 30)


while True:
    # Capture a frame
    cv_image = take_photo()

    # Perform text extraction using Tesseract OCR
    text = pytesseract.image_to_string(cv_image)

    # Print the extracted text
    print("Extracted Text:", text)

    # Convert text to speech
    language = 'en'
    audio = gTTS(text=text, lang=language, slow=False)
    audio.save('audio_output.mp3')

    # Display the audio output
    ipd.display(ipd.Audio('audio_output.mp3', autoplay=True))

    # Break the loop if the 'q' key is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


<IPython.core.display.Javascript object>

Extracted Text:  



AssertionError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')